In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Following tutorial: https://medium.com/ilb-labs-publications/fine-tuning-bert-for-a-regression-task-is-a-description-enough-to-predict-a-propertys-list-price-cf97cd7cb98a

In [2]:
import os
# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2024.1 WN/Eecs 448/EECS 448 Group Project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
# Load the autoreload extension
# %load_ext autoreload
# %autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

['Project Brainstorm.gdoc', 'Project Pre-Proposal.gdoc', 'Tasks.gdoc', 'cleaned_wine_training_data.csv', 'Milestone Presentation Outline.gdoc', 'Copy of 448 Project Proposal.gdoc', '448 Project Proposal.gdoc']


In [3]:
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaModel
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset

In [4]:
# Load and preprocess data
print('reading csv...')
data = pd.read_csv(GOOGLE_DRIVE_PATH + "cleaned_wine_training_data.csv")
data = data.dropna(subset=['price'])
print(f"Data shape: {data.shape}")
print('cutting labels...')
# data['price_category'] = pd.cut(data['price'], bins=[0, 20, 50, 100, float('inf')],
#                                  labels=['cheap', 'moderate', 'expensive', 'very expensive'])

reading csv...
Data shape: (93964, 17)
cutting labels...


In [5]:
# X_train, X_val, y_train, y_val = train_test_split(data['description'], data['price_category'],
#                                                   test_size=0.2, random_state=42)
# print('tokenizing...')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
# val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

# # Convert labels to numerical values
# label_map = {'cheap': 0, 'moderate': 1, 'expensive': 2, 'very expensive': 3}
# train_labels = [label_map[label] for label in y_train]
# val_labels = [label_map[label] for label in y_val]

tokenizing...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-base')
encoded_corpus = tokenizer(text = data.description.tolist(),
                           add_special_tokens=True,
                           padding='max_length',
                           truncation='longest_first',
                           max_length=300,
                           return_attention_mask=True)
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']
labels = data.price.to_numpy(dtype=float)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
# train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
#                               torch.tensor(train_encodings['attention_mask']),
#                               torch.tensor(train_labels))

# val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
#                             torch.tensor(val_encodings['attention_mask']),
#                             torch.tensor(val_labels))

# print('creating dataloader...')
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

creating dataloader...


In [6]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels,
                                                                        test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_mask, labels,
                                                 test_size=0.2, random_state=42)

In [7]:
price_scaler = StandardScaler()
price_scaler.fit(train_labels.reshape(-1, 1))
train_labels = price_scaler.transform(train_labels.reshape(-1, 1))
test_labels = price_scaler.transform(test_labels.reshape(-1, 1))

In [15]:
def create_dataloaders(inputs, masks, labels):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    label_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor, label_tensor)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True) # or batch=16
    return dataloader

train_loader = create_dataloaders(train_inputs, train_masks, train_labels)
test_loader = create_dataloaders(test_inputs, test_masks, test_labels)

In [7]:
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
# optimizer = AdamW(model.parameters(), lr=1e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
class RobertaRegressor(nn.Module):
    def __init__(self, drop_rate=0.2):
        super(RobertaRegressor, self).__init__()
        D_in, D_out = 768, 1
        self.roberta = RobertaModel.from_pretrained('FacebookAI/roberta-base')
        self.regressor = nn.Sequential(nn.Dropout(drop_rate),
                                       nn.Linear(D_in, D_out))
        self.double() # or self.double()

    def forward(self, input_ids, attention_masks):
        outputs = self.roberta(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs

model = RobertaRegressor()
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
loss_fn = nn.MSELoss()

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 296.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 49.06 MiB is free. Process 2332 has 14.70 GiB memory in use. Of the allocated memory 14.18 GiB is allocated by PyTorch, and 398.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
# print('training...')
# num_epochs = 3
# for epoch in range(num_epochs):
#     model.train()
#     for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
#         batch = tuple(t.to(device) for t in batch)
#         input_ids, attention_mask, labels = batch
#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         # print(f'Batch {i}: {loss}')

#     # Validation
#     model.eval()
#     val_losses = []
#     val_preds = []
#     val_true = []
#     for i, batch in enumerate(val_loader):
#         batch = tuple(t.to(device) for t in batch)
#         input_ids, attention_mask, labels = batch
#         with torch.no_grad():
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)  # Include labels during inference
#             loss = outputs.loss
#             logits = outputs.logits
#             val_losses.append(loss.item())
#             val_preds.extend(torch.argmax(logits, axis=1).cpu().numpy())
#             val_true.extend(labels.cpu().numpy())
#             print(f'Batch {i}: {loss}')


#     val_accuracy = (sum([1 if p == t else 0 for p, t in zip(val_preds, val_true)]) / len(val_true)) * 100
#     print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {sum(val_losses) / len(val_losses)}, '
#           f'Validation Accuracy: {val_accuracy}%')

training...


  2%|▏         | 72/4699 [00:22<24:03,  3.21it/s]


KeyboardInterrupt: 

In [14]:
def train(model, optimizer, loss_fn, epochs, train_loader, device):
    for epoch in range(epochs):
        print(f"Training Epoch {epoch+1}...")
        model.train()
        # for step, batch in enumerate(train_loader):
        for step, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
            print(step)
            batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)
            loss = loss_fn(outputs.squeeze(), batch_labels.squeeze())
            loss.backward()
            optimizer.step()

    return model

epochs = 3
model = train(model, optimizer, loss_fn, epochs, train_loader, device)

Training Epoch 1...
0


OutOfMemoryError: CUDA out of memory. Tried to allocate 264.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 49.06 MiB is free. Process 2332 has 14.70 GiB memory in use. Of the allocated memory 14.22 GiB is allocated by PyTorch, and 365.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def evaluate(model, loss_fn, test_loader, device):
    model.eval()
    test_loss, test_r2 = [], []
    for batch in test_loader:
        batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks)
        loss = loss_fn(outputs, batch_labels)
        test_loss.append(loss.item())
        r2 = r2_score(outputs, batch_labels)
        test_r2.append(r2.item())
    return test_loss, test_r2

def r2_score(outputs, labels):
    labels_mean = torch.mean(labels)
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - outputs) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def predict(model, data_loader, device):
    model.eval()
    output = []
    for batch in data_loader:
        batch_inputs, batch_masks, _ = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks).view(1, -1).tolist()[0]
    return output